# Jacobinet Tutorial: Computing and Visualizing Gradients Using Backward Models in Keras

In this tutorial, we'll build a simple neural network in Keras and then use the *Jacobinet* library to compute the gradient (Jacobian) of the output with respect to the input. We'll visualize both the forward and backward models and explore how the chain rule applies.

## Step 1: Define the Forward Model

We'll create a simple feedforward neural network with the following architecture:
- Dense Layer (10 units) + ReLU activation
- Dense Layer (1 unit) - Output layer

The model takes a single input of shape `(1,)` and outputs a single value.

```python
# Import necessary libraries
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Input

# Build the forward model
model = Sequential(
    [
        Dense(10, input_shape=(1,), name='Dense1'),
        Activation('relu', name='ReLU1'),
        Dense(1, name='Output'),
    ]
)

# Display model summary
print("### Forward Model Summary")
model.summary()


In [6]:
# Build the model
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Input

model = Sequential(
    [
        Dense(10, input_shape=(3,), name='Dense1'),
        Activation('relu', name='ReLU1'),
        Dense(2, name='Output'),
    ]
)
    
model(Input((3,)))

# Display model summary
print("### Forward Model Summary")
model.summary()

### Forward Model Summary


/Users/ducoffe_m/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Dense1 (Dense)                  │ (None, 10)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ReLU1 (Activation)              │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 2)              │            22 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62 (248.00 B)

 Trainable params: 62 (248.00 B)

 Non-trainable params: 0 (0.00 B)

## Visualize the forward model


In [7]:
dot_img_file = './model_dense.png'
keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True, show_layer_names=True)

from IPython.display import Image, display, HTML

display(HTML('<div style="text-align: center;"><img src="{}" width="400"/></div>'.format(dot_img_file)))


# Step 2: Compute the Backward Model Using JacoBinet

*Jacobinet* allows us to compute a backward model that represents the gradient of the output with respect to the input. This is key to understanding the chain rule in neural networks, which is fundamental in backpropagation.






## Import JacoBinet library


In [8]:
import jacobinet
from jacobinet import clone_to_backward

## Get the backward model using JacoBinet

In [9]:
backward_model = clone_to_backward(model)


In [10]:
backward_model

<BackwardModel name=backward_model, built=True>

## Display backward model summary


In [11]:
print("### Backward Model Summary")
backward_model.summary()

### Backward Model Summary


Model: "backward_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_8       │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ backward_dense_1    │ (1, 10)           │         22 │ input_layer_9[0]… │
│ (BackwardDense)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense1 (Dense)      │ (None, 10)        │         40 │ input_layer_8[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ backward_activation │ (1, 10)           │          0 │ backward_dense_1… │
│ (BackwardActivatio… │                   │            │ Dense1[1][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ backward_dense      │ (1, 3)            │         40 │ backward_activat… │
│ (BackwardDense)     │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 62 (248.00 B)

 Trainable params: 62 (248.00 B)

 Non-trainable params: 0 (0.00 B)

In [12]:
dot_img_file_backward = './model_dense_backward.png'
keras.utils.plot_model(backward_model, to_file=dot_img_file_backward, show_shapes=True, show_layer_names=True)

display(HTML('<div style="text-align: center;"><img src="{}" width="800"/></div>'.format(dot_img_file_backward)))


## Step 3: Understanding the Chain Rule in Neural Networks

In a neural network, the **chain rule** is fundamental for propagating gradients backward, from the output layer to the input layer. It allows us to compute the gradient of the loss with respect to each parameter by combining partial derivatives at each layer. Specifically, The gradient given the input  can be split using the chain rule along any latent dimension *h* given the following expression:

$$ 
\frac{\partial L}{\partial x_i} = \frac{\partial L}{\partial h} \cdot \frac{\partial h}{\partial x_i} 
$$

In this tutorial, the loss function is simply the identity function applied to the single output of the network, meaning the output itself acts as the loss $ L(y) = y$. This simplification helps illustrate how the gradients flow through the network without additional complexity from the loss computation itself.


- $\frac{\partial L}{\partial x_i}$: Gradient of the loss \(L\) with respect to the input \(x_i\).
- $\frac{\partial L}{\partial h} $: Gradient of the loss with respect to a latent dimension \(h\).
- $\frac{\partial h}{\partial x_i}$: Gradient of the latent ouput \(h\) with respect to the input \(x_i\).

### How the Backward Model Works

The **backward model** computes the **Jacobian matrix**, which represents all the partial derivatives of the output with respect to the input. This is crucial in backpropagation since it helps propagate the gradients back through the network.

In simpler terms, backpropagation computes how much the network's input contributed to the final loss by applying the chain rule across all layers.

### Visualizing the Flow of Gradients

The forward model computes the predictions. The backward model, on the other hand, tells us how sensitive the output is to changes in the input by tracing gradients backward through the layers.

### Conclusion

Using the Jacobinet library, we have successfully visualized the backward propagation model. This backward model helps us understand how gradients flow through the network.
